In [8]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, TimeDistributed
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [9]:
# Define paths
dataset_path = "/content/dataset/FF++"
categories = ["fake", "real"]

In [10]:
# Define video frame processing parameters
IMG_SIZE = (64, 64)
SEQUENCE_LENGTH = 20

def load_videos():
    data, labels = [], []
    for category in categories:
        label = 1 if category == "fake" else 0
        folder_path = os.path.join(dataset_path, category)
        for video_file in os.listdir(folder_path):
            video_path = os.path.join(folder_path, video_file)
            frames = preprocess_video(video_path)
            data.append(frames)
            labels.append(label)
    return np.array(data), np.array(labels)

# Function to preprocess video frames
def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < SEQUENCE_LENGTH:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, IMG_SIZE)
        frame = frame / 255.0  # Normalize pixel values
        frames.append(frame)
    cap.release()

    if len(frames) < SEQUENCE_LENGTH:
        frames += [np.zeros_like(frames[0])] * (SEQUENCE_LENGTH - len(frames))  # Padding
    return np.array(frames)

# Load data
data, labels = load_videos()

In [11]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)



In [12]:
# Define CNN-LSTM model
def create_cnn_lstm_model(input_shape):
    model = Sequential([
        TimeDistributed(Conv2D(32, (3,3), activation='relu', padding='same'), input_shape=input_shape),
        TimeDistributed(MaxPooling2D((2,2))),

        TimeDistributed(Conv2D(64, (3,3), activation='relu', padding='same')),
        TimeDistributed(MaxPooling2D((2,2))),

        TimeDistributed(Flatten()),

        LSTM(64, return_sequences=False),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



In [13]:
# Train the model
input_shape = (SEQUENCE_LENGTH, IMG_SIZE[0], IMG_SIZE[1], 3)
model = create_cnn_lstm_model(input_shape)
model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 176s 4s/step - accuracy: 0.5410 - loss: 0.7326 - val_accuracy: 0.5500 - val_loss: 0.6910
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 194s 4s/step - accuracy: 0.5134 - loss: 0.7173 - val_accuracy: 0.4875 - val_loss: 0.7023
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 201s 4s/step - accuracy: 0.6041 - loss: 0.6698 - val_accuracy: 0.5875 - val_loss: 0.6920
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 217s 4s/step - accuracy: 0.6711 - loss: 0.6179 - val_accuracy: 0.5750 - val_loss: 0.6960
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 160s 4s/step - accuracy: 0.7343 - loss: 0.5528 - val_accuracy: 0.6000 - val_loss: 0.6804
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 200s 4s/step - accuracy: 0.8048 - loss: 0.4368 - val_accuracy: 0.6625 - val_loss: 0.6975
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 167s 4s/step - accuracy: 0.7843 - loss: 0.4229 - val_accuracy: 0.6125 - val_loss: 0.7314
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 160s 4s/step - accuracy: 0.8172 - loss: 0.3838 - val_accuracy: 0.6625 - v

In [14]:
# Model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, 20, 64, 64, 32)      │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 20, 32, 32, 32)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, 20, 32, 32, 64)      │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_3 (TimeDistributed) │ (None, 20, 16, 16, 64)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_4 (TimeDistributed) │ (None, 20, 16384)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │       4,210,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,703,685 (48.46 MB)

 Trainable params: 4,234,561 (16.15 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 8,469,124 (32.31 MB)

In [15]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.6938 - loss: 0.8426
Test Accuracy: 0.6375
